### <center> <h1>Systèmes de Décision et Préférences : Projet</h1> <center>
###### <center> <h2> Amine Larhchim, Gauthier Roy, Agathe Gioan </h2><center>

# Sommaire

* [1. Définitions du sujet](#chapter1)
    * [1.1. Contexte](#section_1_1)
    * [1.2. Données](#section_1_2)
    * [1.3. Variables de décision](#section_1_3)
    * [1.4. Fonctions et Objectifs](#section_1_4)
    * [1.5. Contraintes](#section_1_5)
* [2. Structure du code pour les données du problème](#chapter2)
* [3. Résolution du problème](#chapter3)
    * [3.1. Solutions non dominées](#section_3_1)
    * [3.2. Modèle de préférence](#section_3_2)
* [4. Résultats](#chapter4)

### 1. Définitions du sujet <a class="anchor" id="chapter1"></a>

#### 1.1. Contexte <a class="anchor" id="section_1_1"></a>

La société CompuOpti implémente des solutions d'optimisation et aide à la décision pour leurs clients. <br>
Chaque projet nécessite de staffer sur un certains nombre de jours, un certains nombres d'employés sur des compétences spécifiques. <br>
Le but est de fournir une planification optimale du personnel et les affectations sur les différents projets. <br><br>

Pour cela, il faudra prendre en compte un horizon de temps sur lequel se déroulent les projets, le fait que chaque employé possède un certain nombre de qualifications parmi un ensemble donné, ont des jours de congés prédéfinis. <br>
Chaque projet fait appel à certaines qualifications sur un certains nombres de jours et produit un gain s'il est réalisé avant sa date de livraison prédéfinie, ou bine il y aura des pénalités financières. <br><br> 
Les critères d'optimalité sont multiples : <br> 
 - En premier lieu, nous voulons maximiser le profit. 
 - Nous souhaitons ensuite que le nombre de projets par employés soit minimal. 
 - Les projets doivent être réalisés dans un nombre limités de jours consécutifs. 
 - Un employé ne peut réalisé qu'un projet et n'utiliser qu'une qualification (qu'il possède) à la fois, et ne doit pas travailler pendant ses jours de congés. 
 - Un projet n'est réalisé que si tous les jours de travail dédiés à chacune de ses qualifications on été couvert dans l'horizon de temps. Il ne peut être réalisé qu'une seule fois. 

#### 1.2. Données <a class="anchor" id="section_1_2"></a>

Pour tester notre modèle, nous travaillons sur trois jeux de données de tailles différentes. Chaque jeux nous donnent des données différentes au format suivant. <br><br>

Le json constistue un dictionnaire de données dans lequel nous retrouvons : 
- l'horizon total
- l'ensemble des qualifications possibles
- un dictionnaire pour les employés
- un dictionnaire pour les projets 

Le dictionnaire pour les employés contient, pour chaque employé, les informations suivantes :
- le nom de l'employé 
- ses qualifications
- ses jours de congé

Le dictionnaire pour les projets contient, pour chaque projet, les informations suivantes :
- le nom du projet
- son gain 
- sa date de livraison 
- sa pénalité par jour de retard 
- le nombre de jours nécessaires par qualification

Voici un exemple de petit jeu de données qsur lequel nous pourrions travailler: <br><br>
{<br>
&emsp;    "horizon": 5,<br>
&emsp;    "qualifications": ["A","B","C"],<br>
&emsp;    "staff": [<br>
&emsp;        &emsp;{<br>
&emsp;        &emsp;    &emsp;"name": "Olivia",<br>
&emsp;        &emsp;    &emsp;"qualifications": ["A","C"],<br>
&emsp;        &emsp;    &emsp;"vacations": []<br>
&emsp;        &emsp;},<br>
&emsp;        &emsp;{<br>
&emsp;        &emsp;    &emsp;"name": "Liam",<br>
&emsp;        &emsp;    &emsp;"qualifications": ["A","B"],<br>
&emsp;        &emsp;    &emsp;"vacations": [1]<br>
&emsp;        &emsp;}<br>
&emsp;    ],<br>
&emsp;    "jobs": [<br>
&emsp;        &emsp;{<br>
&emsp;        &emsp;    &emsp;"name": "Job1",<br>
&emsp;        &emsp;    &emsp;"gain": 20,<br>
&emsp;        &emsp;    &emsp;"due_date": 3,<br>
&emsp;        &emsp;    &emsp;"daily_penalty": 3,<br>
&emsp;        &emsp;    &emsp;"working_days_per_qualification": {"A": 1,"B": 1,"C": 1}<br>
&emsp;        &emsp;},<br>
&emsp;        &emsp;{<br>
&emsp;        &emsp;    &emsp;"name": "Job2",<br>
&emsp;        &emsp;    &emsp;"gain": 15,<br>
&emsp;        &emsp;    &emsp;"due_date": 3,<br>
&emsp;        &emsp;    &emsp;"daily_penalty": 3,<br>
&emsp;        &emsp;    &emsp;"working_days_per_qualification": {"A": 1,"B": 2}<br>
&emsp;        &emsp;}<br>
&emsp;    ]<br>
&emsp;}
&emsp;

In [ ]:
Voir code 
ajouter tous les MVAR 

#### 1.3. Paramètres du problème <a class="anchor" id="section_1_4"></a>

Définition des paramètres du problème et leurs notations : 

Paramètre d'une instance:
- $ T={1,...,t} $ l'horizon des temps
- $ Q={1,...,q} $ l'ensemble des compétences
- $ I={1,...,i} $ l'ensemble des employés
- $ P={1,...,p} $ l'ensemble des employés <br>

Un employé $i \in I$ est caractérisé par :
- $ qualifications_i $ ces qualifications.
- $ vacations_i$ ces jours de congés. <br>

Un projet $p \in P$ est caractérisé par:
- $ DueDate_p $ sa date de rendu attendu.
- $ requirement_{p,q}$ pour $q \in Q$ la quantité de compétence q attendue pour ce projet.
- $Gain_p$ le gain obtenu en réalisant le projet.
- $Penality_p$ la pénalité par jour de retard. <br>



#### 1.4. Variables de décision <a class="anchor" id="section_1_3"></a>

- $X_{i,p,t,q}$ binaire : « vaut 1 si l'employé i est staffé sur le projet p au jour t avec la compétence q et 0 sinon» <br>
- $Y_{p} $ binaire : « vaut 1 si le projet p est réalisé et 0 sinon. » <br>
- $StartDate_{p} \in T$ date de début de réalisation du projet<br>
- $EndDate_{p} \in T$ date de fin de réalisation du projet.



Pour simplifier la formulation du problème nous définissons aussi les variables suivantes :
- $MaxDuration= \max_{\substack{p}}(EndDate_p-startDate_p)$  donne la durée de réalisation du plus long projet. <br> 
- $MaxProject= \max_{\substack{i}} card(p, ∃ t,q,p,  X_{i,p,t,q}=1)$  le nombre de projet sur lesquels est staffé l'employé le plus productif. <br>
- $PenalityFee_p= Penality_p\times(EndDate_p-DueDate_p)\times Y_p$


	

##### Problème : <a class="anchor" id="section_1_3"></a>

Maximize $\sum_{\substack{p}}(Gain_p-PenalityFee_p)\times Yp$ <br>
Minimize $MaxDuration$	<br>
Minimize $MaxProject$

#### 1.5. Contraintes du problème <a class="anchor" id="section_1_5"></a>

Contrainte de qualification du personnel : <br>
&emsp; ∀i, ∀q not in qualifications(i), $\sum_{\substack{p,t}} X_{i,p,t,q}=0$	<br><br>
Contrainte de congé : <br>
&emsp; ∀i,t in vacations(i), $\sum_{\substack{p,q}} X_{i,p,t,q}=0$ <br><br>
Contrainte d’unicité de l’affectation quotidienne du personnel : <br>
&emsp; ∀t,i $\sum_{\substack{p,q}} X_{i,p,t,q} <= 1$ <br><br>
Contrainte de couverture des qualifications du projet : <br>
&emsp; ∀p,q, $\sum_{\substack{i,t}} X_{i,p,t,q} <= qualification(p,q)$ <br>
&emsp; $\sum_{\substack{i,t,q}}Y_{p} * X_{i,p,t,q}>=qualifications(p,q) $


### 2.  Résolution du problème <a class="anchor" id="chapter3"></a>

Après avoir défini le problème sous forme d'un problème d'optimisation multiobjectif. Nous allons à présent chercher les solutions non-dominés. 
Nous utilisons la méthode Epsilon constrain afin d'obtenir les points nadir pour le 2ème et 3ème objectifs. 

On limite notre espace au point inférieurs aux points nadir pour les objectifs 2 et 3. Nous optimisions la solution du première objectif en fixant la durée maximum pour un projet et le nombre maximum de projets sur lesquels est staffé un employé dans notre espace.
On filtre les résultats obtenus en vérifiant qu'ils sont bien des solutions non-dominés. 

#### 2.1.Epsilon constraint <a class="anchor" id="section_3_1"></a> 

#### 3.2. Points nadir <a class="anchor" id="section_3_1"></a> 

#### 2.1. Solutions non-dominées<a class="anchor" id="section_3_1"></a> 
    

### 4. modèle de préférence  <a class="anchor" id="section_3_2"></a> 

A l'étape précédente nous avons détérminer les plannings correspondant à l'ensemble des solutions non dominés du problème multicritère (P). <br>

Pour pouvoir prendre une décision par rapport à notre problème, il va falloir discriminer les solutions entre elles et pouvoir juger de la qualité de chacun des plannings.
Pour chaque solutions on a un triplet $(x_1,x_2,x_3)$ avec $x_1$ le gain correspondants à l'ensemble des projets réalisés, $x_2$ la durée du projet le plus long et $x_3$ le nombre maximum de projets différents sur lequels travaille le même employé. <br>

Il semble évident que l'objectif le plus important est financier. On va d'abord chercher à avoir le plannings le plus lucratif. Mais on veut aussi ne pas trop surcharger nos équipes ou passer trop de temps sur un seul et même projet. Chaque décideur a sa propre sensiblité à chacun des objectifs qu'il est difficile de modéliser formellement. Nous proposons donc d'inférer ses préférences avec le modèle UTA. <br>

Pour choisir parmis ces solutions nous allons élaborer un modèle de préférence à partir d'exemples de plannings : incorrects, corrects et satisfaisants choisis par le décideur.

### 4.  Résultats <a class="anchor" id="chapter4"></a>

In [ ]:
Résultas des 3 instances 
